In [4]:
# import paddle
# from paddle import utils
!pip install paddle --upgrade
!pip install paddlepaddle

In [2]:
!pip install utils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=795ec60324487b177ea79838d7d6e0601c2f8f4d46eebddbbeb28165d468e129
  Stored in directory: c:\users\00147515\appdata\local\pip\cache\wheels\4c\a5\a3\ab48e06c936b39960801612ee2767ff53764119f33d3d646e7
Successfully built utils


In [1]:
import numpy as np
import pandas as pd
import re
import os
import urllib3
from fuzzywuzzy import fuzz
from itertools import tee, islice
from paddle_pdf import save_pages,convert_paddle_ds
from news_language_extraction import article_extraction,lang_detect
import newspaper
import csv
import json
from ftlangdetect import detect
from langcodes import tag_is_valid, Language
from bs4 import BeautifulSoup
import requests
import pandas as pd


C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


ModuleNotFoundError: No module named 'paddle.utils'

In [3]:
import pprint
from PIL import Image

In [ ]:

def generate_ngrams(text, n):
    # Function to generate n-grams from a text
    if not text:
        return []
    words = text.split()
    ngrams = zip(*[islice(seq, index, None) for index, seq in enumerate(tee(words, n))])
    return [' '.join(ngram) for ngram in ngrams]


def calculate_max_similarity(name, article):
    # Calculate maximum similarity score using various fuzzy matching methods
    if not name:  # If name is None or empty
        return 0
    
    max_similarity = 0
    if isinstance(name, str):  # Check if name is a valid string
        name_length = len(name.split())
        article_ngrams = generate_ngrams(article, name_length)
        
        for ngram in article_ngrams:
            # Calculate similarity scores
            ratio = fuzz.ratio(name, ngram)
            # partial_ratio = fuzz.partial_ratio(name, ngram)
            token_set_ratio = fuzz.token_set_ratio(name, ngram)
            token_sort_ratio = fuzz.token_sort_ratio(name, ngram)
            
            # Get maximum similarity score
            max_score = max(ratio, token_set_ratio, token_sort_ratio)
            
            # Update max_similarity if current score is higher
            if max_score > max_similarity:
                max_similarity = max_score
    
    return max_similarity


def clean_wlname(wlname):
    # Clean and preprocess wlname
    wlname=wlname[0]
    wlname=wlname.upper()
    unwanted_substrings = [ 'A/L','A/P','S/O','D/O','SO','DO','AP','BINTI','BINTE','B','BTE','BT','IBNI','IBNU','ANAK','\'','-','HAJI','HAJJAH','HAJAH','HJH','HJ','ALHAJ','HADJI','WIRA','PANGLIMA','IR','DR.','PROF.','SR','K.D.Y.T.M.','K.D.Y.M.M.','D.Y.M.M.','D.Y.T.M.','Y.T.M.','D.Y.A.M.','Y.M.M.','Y.A.M.','Y.BHG.','KDYTM','KDYMM','DYMM','DYTM','DYAM','YMM','YTM','YANG  DI PERTUAN  AGONG','DULI  YANG  MAHA  MULIA  SERI  PADUKA  BAGINDA','TIMBALAN  YANG  DI PERTUAN  AGONG','YANG DI PERTUAN BESAR','YANG  DI PERTUA  NEGERI','SULTAN','RAJA  PERMAISURI  AGONG','TUNKU  AMPUAN  BESAR','TUNKU  AMPUAN','TUNKU  PUAN  BESAR','SULTANAH','TENGKU  AMPUAN  BESAR','TENGKU  AMPUAN  MAHKOTA','TENGKU  AMPUAN','TENGKU  BESAR','TENGKU  MAHKOTA','PADUKA  BONDA  RAJA','RAJA  PERMAISURI','PERMAISURI','RAJA  MUDA','RAJA  PUAN  MUDA','RAJA  PUAN  BESAR','RAJA  DI HILIR','TENGKU  PUAN  MUDA','TENGKU  PUAN','YANG  DI-PERTUAN  MUDA','YANG  DI-PERTUAN  BESAR','TUANKU','TUNKU','TUN','RAJA','TENGKU','MEK','NIK','TUAN','WAN','PUTERI','PUTRI','AWANG','ABANG','DAYANG','CIK  PUAN  BESAR','CIK  PUAN','ENCHE  BESAR','CHE  PUAN  BESAR','SYED','SY','SAYED','SHARIFAH','SH','SYARIFAH','TAN  SRI','DATUK  SERI','DATUK  SRI','DATUK','DATOK','DATO  SRI','DATO  SERI','DATO','DATIN','PEHIN','YBHG','YAB','YB','YANG  AMAT  BERHORMAT','YANG  BERHORMAT','YANG  BERBAHAGIA']
    
    # Create a regular expression pattern that matches any of the unwanted substrings
    pattern = r'\b(?:' + '|'.join(map(re.escape, unwanted_substrings)) + r')\b'
    
    # Remove the unwanted substrings from the names
    wlname = re.sub(pattern, '', wlname).strip()

    # Define the unwanted characters as a list
    unwanted_chars = [',','~','!','#','\$','%','\^','&','\*','_','-','\+','=','\}','\{','\]','\[','\|','\\\\',':',';','\"','<','>','\?','\.']
    
    # Construct the regular expression pattern to remove unwanted characters
    unwanted_pattern = '|'.join(map(re.escape, unwanted_chars))
    
    # Remove unwanted characters and ensure no extra spaces are introduced
    wlname = re.sub(unwanted_pattern, '', wlname).strip()
    
    # Replace specific terms in the wlname
    replacements = {
        'MUHAMMAD': 'MUHD', 'MUHAMMED': 'MUHD', 'MUHAMAD': 'MUHD', 'MUHAMED': 'MUHD', 'MUHD': 'MUHD',
        'MOHAMMAD': 'MOHD', 'MOHAMMED': 'MOHD', 'MOHAMAD': 'MOHD', 'MOHAMED': 'MOHD', 'MOHD': 'MOHD',
        'ABDUL': 'ABD', 'ABDOUL': 'ABD', 'ABDUR': 'ABD', 'ABDOL': 'ABD', 'ABDOOL': 'ABD', 'ABD': 'ABD', 'AB': 'ABD',
        'AHMAD': 'AHMAD', 'AHMED': 'AHMAD', 'AHMET': 'AHMAD',
        'NOOR': 'NOOR', 'NOR': 'NOOR', 'NUUR': 'NOOR', 'NUR': 'NOOR',
        'ABDULLAH': 'ABDULLAH', 'ABDOLLAH': 'ABDULLAH',
        'YUSOFF': 'YUSOFF', 'YUSOF': 'YUSOFF', 'YUSUFF': 'YUSOFF', 'YUSUF': 'YUSOFF'
    }
    
    for term, replacement in replacements.items():
        wlname = re.sub(term, replacement, wlname)
    
    # Remove accents and other diacritics
    wlname = re.sub("[ÁÄÂÃ]", "A", wlname)
    wlname = re.sub("[Å]", "AA", wlname)
    wlname = re.sub("[ÉËÊÈ]", "E", wlname)
    wlname = re.sub("[ÍÏÎ]", "I", wlname)
    wlname = re.sub("[ÓÔÖÕ]", "O", wlname)
    wlname = re.sub("[ÚÜÙ]", "U", wlname)
    wlname = re.sub("[Ø]", "OE", wlname)
    wlname = re.sub("[Ç]", "C", wlname)
    wlname = re.sub("[Ñ]", "N", wlname)
    wlname = re.sub("[Ý]", "Y", wlname)
    wlname = re.sub("[^A-Z\s ]+", "", wlname).strip()
    
    return wlname


def process_wlname_and_article(wlname, article):
    # Main function to process wlname and article
    wlname = clean_wlname(wlname)

    # Extract and clean alias names from parentheses, '@', and '/'
    alias_name2 = re.search(r'\((.*?)\)', wlname)
    alias_name2 = alias_name2.group(1).strip() if alias_name2 else ''

    wlname = re.sub(r'\(.*?\)', '', wlname).strip()

    if '@' in wlname:
        alias_name3 = wlname.split('@')[1].strip()
        wlname = wlname.split('@')[0].strip()
    else:
        alias_name3 = ''

    if '/' in wlname:
        alias_name4 = wlname.split('/')[1].strip()
        wlname = wlname.split('/')[0].strip()
    else:
        alias_name4 = ''
    
    # Calculate similarity scores for wlname and aliases
    similarity1 = calculate_max_similarity(wlname, article)
    similarity2 = calculate_max_similarity(alias_name2, article)
    similarity3 = calculate_max_similarity(alias_name3, article)
    similarity4 = calculate_max_similarity(alias_name4, article)
    
    # Return the maximum similarity score
    max_similarity = max(similarity1, similarity2, similarity3, similarity4)
    
    return max_similarity

def download_file(response,headers, save_as):
    # response = requests.get(url, headers=headers, timeout=10)
    with open(save_as, 'wb') as file:
        file.write(response.content)
        
def similarity_list(wlname,text_extracted):
    Name_relevance_similarity = []
    if len(text_extracted)>5:
        max_similarity = process_wlname_and_article(wlname, text_extracted)
        # print(wlname,max_similarity)
        Name_relevance_similarity.append(str(max_similarity))
    else:
        Name_relevance_similarity.append('0')
    # print(Name_relevance_similarity)
    return Name_relevance_similarity
     
# Example usage
# wlname = ['HARIYANTO']
# article = '''WEBSarawak Economy. Sarawak is one of the two Malaysian states situated in the northwest of Borneo. For decades, LNG and petroleum have been the primary sources of the â€¦'''

# print(f"Maximum similarity score: {max_similarity}")


In [ ]:

if __name__ == "__main__":
    import sys
    parent_dir = '/app/Name_Screening/Relevance_Checking/anews_relavance'
    headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_agent}.0.0.0 Safari/537.36 Edg/{edge_agent}.0.0.0"
    }
    # df = pd.read_excel('/app/pruthvi_paddle_integ/POC_TMNS/src/Translation_scraping/links_to_re-run.xlsx')
    df = pd.read_csv('/app/Name_Screening/Relevance_Checking/ind_sl_anews1.csv')
    
    '''
    df['Extracted_Text'] = ''
    df['Extracted_Text_1'] = ''
    df['language_detect_1'] = ''
    df['Extracted_Text_2'] = ''
    df['language_detect_2'] = ''
    df['language_detect'] = ''
    df['Status_Code'] = ''
    print(df.head(6))
    
    df['Extracted_Text'] = ''
    df['language_detect'] = ''
    
    print(df.shape[0])
    df.dropna(subset=['V_SOURCE_LINKS'], inplace = True)
    df = df.drop_duplicates(keep='first')
    count = df['V_INFO_SOURCE'].value_counts()
    print(count)
    df_pepex = df[df['V_INFO_SOURCE'] == 'PEPEX']
    df_sanex = df[df['V_INFO_SOURCE'] == 'SANEX']
    df_anews = df[df['V_INFO_SOURCE'] == 'ANEWS']
    # print(df.shape[0])
    df_pepex.to_csv('ind_sl_pepex.csv',index = False)
    df_sanex.to_csv('ind_sl_sanex.csv',index = False)
    df_anews.to_csv('ind_sl_anews.csv',index = False)
    '''
    
    for i in range(0,df.shape[0]):
        # print(i)
        if type(df['V_SOURCE_LINKS'].iloc[i]) == type('str'):
            list_articles = df['V_SOURCE_LINKS'].iloc[i].replace('\n', '').split(' ')
            wlname = []
            if type(df['V_WATCHLIST_PARTY_NAME'].iloc[i]) == type('str'):
                wlname.append(df['V_WATCHLIST_PARTY_NAME'].iloc[i])
            else:
                wlname = ['']
            article_extracted_list_pkg = []
            article_extracted_list_1 = []
            article_extracted_list_2 = []
            article_extracted_language_pkg = []
            article_extracted_language_1 = []
            article_extracted_language_2 = []
            url_status_code = []
            Name_relevance_similarity_1 = []
            Name_relevance_similarity_2 = []
            Name_relevance_similarity_pkg = []
            max_similarity= 0
            # print(list_articles,type(list_articles))
            a = 0
            for j in list_articles:
                a = a+1
                print(j)
                text_extracted = ['']
                text_extracted_1 = ['']
                text_extracted_2 = ['']
                language_extracted = ['']
                
                try:
                    
                    article_html1 = requests.get(j, headers=headers, timeout=10)
                    print(article_html1.status_code)
                    content_type = article_html1.headers.get('content-type')
                    print(content_type)
                    
                    if article_html1.status_code!=200:
                        content_type = 'text/html'
                        print(content_type)
                    folder_name = '{}_{}_{}'.format(df['N_PARTY_WATCHLIST_MATCH_ID'].iloc[i],df['V_INFO_SOURCE'].iloc[i],a)
                    path_folder = os.path.join(parent_dir, folder_name)
                    if 'text/html' in content_type:
                        text_extracted,text_extracted_1,text_extracted_2 = article_extraction(j,article_html1)
                        # print(text_extracted)
                        article_extracted_list_pkg.append(text_extracted)
                        article_extracted_list_1.append(text_extracted_1)
                        article_extracted_list_2.append(text_extracted_2)
                        
                        article_extracted_language_pkg.append(lang_detect(text_extracted[0]))
                        article_extracted_language_1.append(lang_detect(text_extracted_1[0]))
                        article_extracted_language_2.append(lang_detect(text_extracted_2[0]))
                        # similarity_list(wlname,text_extracted)
                        Name_relevance_similarity_1.append(similarity_list(wlname,text_extracted_1[0]))
                        Name_relevance_similarity_2.append(similarity_list(wlname,text_extracted_2[0]))
                        Name_relevance_similarity_pkg.append(similarity_list(wlname,text_extracted[0]))
                        print('############################################')
                        # print(article_html1.status_code)
                    elif 'image' in content_type:
                        if not os.path.exists(path_folder):
                            os.mkdir(path_folder)
                        file_name = '{}_image_{}.tiff'.format(df['N_PARTY_WATCHLIST_MATCH_ID'].iloc[i],a)
                        path_image = os.path.join(path_folder, file_name)
                        download_file(article_html1,headers,path_image)
                        # text_extracted = []
                        text_extracted = save_pages(path_image)
                        # print(text_extracted)
                        article_extracted_list_pkg.append(text_extracted)
                        article_extracted_list_1.append(['Image'])
                        article_extracted_list_2.append(['Image'])
                        article_extracted_language_pkg.append(lang_detect(text_extracted[0]))
                        article_extracted_language_1.append(['Image'])
                        article_extracted_language_2.append(['Image'])
                        Name_relevance_similarity_1.append(['0'])
                        Name_relevance_similarity_2.append(['0'])
                        Name_relevance_similarity_pkg.append(similarity_list(wlname,text_extracted[0]))
                    elif 'pdf' in content_type:
                        if not os.path.exists(path_folder):
                            os.mkdir(path_folder)
                        file_name = '{}_pdf_{}.pdf'.format(df['N_PARTY_WATCHLIST_MATCH_ID'].iloc[i],a)
                        path_pdf = os.path.join(path_folder, file_name)
                        # print(path_pdf)
                        download_file(article_html1,headers,path_pdf)
                        # text_extracted = []
                        text_extracted = save_pages(path_pdf)
                        # text_extracted = save_pages(path_pdf)
                        # print(text_extracted)
                        # text_extracted(article_html1)
                        article_extracted_list_pkg.append(text_extracted)
                        article_extracted_list_1.append(['pdf'])
                        article_extracted_list_2.append(['pdf'])
                        article_extracted_language_pkg.append(lang_detect(text_extracted[0]))
                        article_extracted_language_1.append(['pdf'])
                        article_extracted_language_2.append(['pdf'])
                        Name_relevance_similarity_1.append(['0'])
                        Name_relevance_similarity_2.append(['0'])
                        Name_relevance_similarity_pkg.append(similarity_list(wlname,text_extracted[0]))
                    url_status_code.append(str(article_html1.status_code))
                except Exception as e:
                    print(e)
                    print('********************************************')
                    article_extracted_list_pkg.append(text_extracted)
                    article_extracted_language_pkg.append(language_extracted)
                    article_extracted_list_1.append(text_extracted_1)
                    article_extracted_list_2.append(text_extracted_2)
                    article_extracted_language_1.append(language_extracted)
                    article_extracted_language_2.append(language_extracted)
                    Name_relevance_similarity_1.append(['0'])
                    Name_relevance_similarity_2.append(['0'])
                    Name_relevance_similarity_pkg.append(['0'])
                    url_status_code.append('')
    
            # df['Extracted_Text'].iloc[i]
            # print(article_extracted_list_pkg)
            # print(df.dtypes)
            for column in df.columns:
                if df[column].dtype == 'float64':
                    df[column] = df[column].astype('object')
            # print(df.dtypes)
            df.at[i,'Extracted_Text_pkg'] = article_extracted_list_pkg
    
            df.at[i,'language_detect_pkg'] = article_extracted_language_pkg
            df.at[i,'Name_relevance_similarity_pkg'] = Name_relevance_similarity_pkg
            df.at[i,'Name_relevance_similarity_1'] = Name_relevance_similarity_1
            df.at[i,'Name_relevance_similarity_2'] = Name_relevance_similarity_2
            
            # print(url_status_code)
            # print(df.dtypes)
            
            # print(df.dtypes)
            df['Status_Code'] = df['Status_Code'].astype('object')
            # df.set_value(i, 'Status_Code', url_status_code)
            df.at[i,'Status_Code'] = url_status_code
            df.at[i,'Extracted_Text_1'] = article_extracted_list_1
            df.at[i,'language_detect_1'] = article_extracted_language_1
            df.at[i,'Extracted_Text_2'] = article_extracted_list_2
            df.at[i,'language_detect_2'] = article_extracted_language_2
            
            # print(i)
            if (i%500)==0:
                df.to_csv('/app/Name_Screening/Relevance_Checking/anews_relavance/ind_sl_anews1_language.csv',index=False)
        else:
            pass
    
    df.to_csv('/app/Name_Screening/Relevance_Checking/anews_relavance/ind_sl_anews1_language.csv',index=False)
